In [1]:
import sctk

In [2]:
import anndata as ad

# starting from the data after clean_sc_counts
data_dir = "/Users/arturszalata/PycharmProjects/task-dge-perturbation-prediction/resources/neurips-2023-data"

In [61]:
sc_counts = ad.read_h5ad(f'{data_dir}/sc_counts_cleaned.h5ad')

In [62]:
sctk.calculate_qc(sc_counts, flags={"mito": r"^MT-", "ribo": r"^RP[LS]"})

In [63]:
sctk.cellwise_qc(sc_counts)

n_counts: [2358.802254718472, 29992.009765625], 299272/301785 passed
n_genes: [861.7906960906522, 6238.99853515625], 301402/301785 passed
percent_mito: [0.025329280644655228, 19.193623063602075], 301119/301785 passed
percent_ribo: [6.9978885607387955, 50.10729574034552], 295113/301785 passed
291883/301785 pass


In [64]:
sc_counts = sc_counts[sc_counts.obs.cell_passed_qc]

In [65]:
sc_counts.obs['cell_type'] = sc_counts.obs['cell_type'].apply(lambda x: 'T cells' if  x.startswith("T ") else x)

/var/folders/98/9ygjwkpd12d1vxydfprn1st00000gp/T/ipykernel_1812/574505899.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  sc_counts.obs['cell_type'] = sc_counts.obs['cell_type'].apply(lambda x: 'T cells' if  x.startswith("T ") else x)


In [66]:
sc_counts.obs["library_well"] = sc_counts.obs["library_id"].astype(str) + "_" + sc_counts.obs["well"].astype(str)

In [67]:
sc_counts[sc_counts.obs["library_well"].map(sc_counts.obs["library_well"].value_counts()) < 200].shape[0]

719

In [68]:
# remove samples that belong to library_well groups with less than 200 cells
sc_counts = sc_counts[sc_counts.obs["library_well"].map(sc_counts.obs["library_well"].value_counts()) >= 200]

In [69]:
sc_counts.shape[1]

21265

In [70]:
# filter out genes expressed in less than 100 cells per plate

In [71]:
import numpy as np
import pandas as pd

plates = sc_counts.obs['plate_name'].unique()

# Step 2: Initialize a DataFrame to store counts
gene_counts_per_plate = pd.DataFrame(index=sc_counts.var_names, columns=plates, dtype=int)

# Step 3: Iterate over each plate and calculate expression counts
for plate in plates:
    # Subset the AnnData object for the current plate
    subset = sc_counts[sc_counts.obs['plate_name'] == plate]
    
    # Calculate expression counts (genes x cells > 0)
    expressed_genes = (subset.X > 0).sum(axis=0)
    
    # Check if the result needs conversion from sparse matrix format
    if isinstance(expressed_genes, np.matrix):
        expressed_genes = np.array(expressed_genes).flatten()
    
    # Store the counts in the DataFrame
    gene_counts_per_plate[plate] = expressed_genes

# Step 4: Aggregate counts across plates (max or sum based on the requirement)
# We use `max` here to find if any gene meets the criteria in at least one plate
max_counts = gene_counts_per_plate.max(axis=1)

# Step 5: Create a mask for genes to keep (genes expressed in at least 100 cells in any plate)
genes_to_keep = max_counts >= 100

# Filter the AnnData object to keep only these genes
sc_counts = sc_counts[:, genes_to_keep]

In [72]:
sc_counts.shape[1]

18110

In [73]:
del sc_counts.uns

In [82]:
# to be replaced with the current filtering code
sc_counts.obs['plate_well_cell_type'] = sc_counts.obs['plate_name'].astype('str') \
    + '_' + sc_counts.obs['well'].astype('str') \
    + '_' + sc_counts.obs['cell_type'].astype('str')
sc_counts.obs['plate_well_cell_type'] = sc_counts.obs['plate_well_cell_type'].astype('category')

In [85]:
(sc_counts.obs["plate_well_cell_type"].value_counts() < 10).sum()

40

In [86]:
sc_counts[sc_counts.obs["plate_well_cell_type"].map(sc_counts.obs["plate_well_cell_type"].value_counts()) < 10].shape[0]

167

In [87]:
sc_counts = sc_counts[sc_counts.obs["plate_well_cell_type"].map(sc_counts.obs["plate_well_cell_type"].value_counts()) > 10]

In [91]:
sc_counts.obs = sc_counts.obs.drop(columns=['n_counts', 'log1p_n_counts', 'n_genes', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_top50', 'cell_passed_qc'])

In [97]:
sc_counts.var = sc_counts.var.drop(columns=['n_counts', 'n_cells'])

In [109]:
del sc_counts.obsm["X_pca"]

In [110]:
del sc_counts.obsm["X_umap"]

In [113]:
sc_counts.write_h5ad("../data/sc_counts_filtered.h5ad", compression='gzip')